# Databases

## Deduplicate
Write a SQL query to find all duplicate email addresses.

```
+--------+----------------------+
| Id     | Email                |
+--------+----------------------+
| 1234   | mpilgrim@example.com |
| 2534   | spilgrim@example.com |
| 36245  | mpilgrim@example.com |
+--------+----------------------+
```

The above table should give a result like this

```
+----------------------+
| Email                |
+----------------------+
| mpilgrim@example.com |
+----------------------+
```

#### Environment
* Developed with Python 3.5.5, installed via Anaconda

### Thought process
* Get a quick course in SQL to answer this
* I am going to go a bit further and create a sample DB in here to get a bit more familiar - have time
* In practice, DB interactions have been abstracted away from me through frameworks. I have prevented duplicate emails by setting the email as a unique constraint
    * e.g. https://stackoverflow.com/a/26529552

Solution below, but honestly I just googled the commands. I forgot semicolons a few times, and was making some mistakes about closing the connection too early (I closed cursors).

#### A summary of my search history:
1. `jupyter sql notebook` - tried finding any existing plugins to run sql nicely in a notebook (for display purposes)
2. `SQlPad` - curious what that was. Looks interesting, but not what I wanted at the time.
3. `python create sql table sqlite` - looking for some examples of sqlite3 being used t create a table.
4. `code block markdown` - for formatting this notebook
5. `email primary key django` - to remember how I managed unique emails in GAVIP
6. `sql insert random values` - wondering if I could spice this up with a generated set of random emails in SQL (easy in python, but wanted to try it in sql)
7. `drop table if exists` - double checking syntax for that to make solution repeatable
8. `sqlite3 tabulate python example` - I have used `tabulate` before to print tables, so checked for an example of an sql result being printed, but turns out it's straightforward.
9. `sql find duplicates` - now I have the pieces in place, find the actual duplicates
10. `sqlite3 database is locked python` - solution wasn't easily repeatable.. and a locked database was expected. Tried closing cursors in different places and committing changes, but a connection close at the end sufficed. Will be looking further into those details after initial solutions are complete.

#### Prerequisite imports and declarations

In [15]:
try:
    import sqlite3
    from tabulate import tabulate
except ImportError:
    print("Pre-requisites are missing, you can install them in a cell by running: '!pip install <name>'")
    raise

conn = sqlite3.connect('ex4_emails.db')
table_name = "email_table"
emails = ['mpilgrim@example.com', 'spilgrim@example.com', 'cpilgrim@example.com', 'cpilgrim@example.com', 'mpilgrim@example.com'] 

#### Setup

In [16]:
sql_comm_droptable = """
DROP TABLE IF EXISTS {table}
"""

sql_comm_createtable = """
CREATE TABLE IF NOT EXISTS {table} (
 id integer PRIMARY KEY,
 email text NOT NULL
);
"""

sql_comm_insertvals = """
INSERT INTO {table}
(email)
VALUES {values};
"""

try:
    c = conn.cursor()
    c.execute(sql_comm_droptable.format(table=table_name))
    c.execute(sql_comm_createtable.format(table=table_name))
    c.execute(sql_comm_insertvals.format(
        table=table_name,
        values=",".join("('%s')" % email for email in emails)
    ))
except Exception as e:
    print(e)    

#### Render the table

In [17]:
c = conn.cursor()
rows = c.execute("SELECT * from %s;" % table_name)
print(tabulate(rows, ['id', 'email'], tablefmt='psql'))

+------+----------------------+
|   id | email                |
|------+----------------------|
|    1 | mpilgrim@example.com |
|    2 | spilgrim@example.com |
|    3 | cpilgrim@example.com |
|    4 | cpilgrim@example.com |
|    5 | mpilgrim@example.com |
+------+----------------------+


#### Solution 

In [18]:
sql_comm_findduplicates="""
SELECT
    email
FROM
    {table}
GROUP BY
    email
HAVING 
    COUNT(*) > 1;
"""

c = conn.cursor()
rows = c.execute(sql_comm_findduplicates.format(table=table_name))
print(tabulate(rows, ['email'], tablefmt='psql'))

+----------------------+
| email                |
|----------------------|
| cpilgrim@example.com |
| mpilgrim@example.com |
+----------------------+


In [19]:
conn.close()